<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  pip install bs4 --quiet

In [2]:
pip install urllib3 --quiet

In [3]:
pip install marshmallow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00


In [4]:
!pip install pandas_ta --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
pip install free-proxy --quiet

  Preparing metadata (setup.py) ... done


In [6]:
import requests
import pandas as pd
import pandas_ta as ta
import math
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from bs4 import BeautifulSoup
from fp.fp import FreeProxy

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:
    root_url = "https://www.tefas.gov.tr"
    info_endpoint = "/api/DB/BindHistoryInfo"
    concurrently = False
    use_Proxy = False
    fon_type = "YAT"
    proxies = None

    @staticmethod
    def get_combobox_items(url, select_id):
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')
        select_element = soup.find('select', id=select_id)

        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        options = select_element.find_all('option')
        options = list(filter(None, options))

        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')

        return items

    def fetch_info(self, fonunvantip, start_date_initial, end_date_initial):
        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
            counter = range_date.days / range_interval
            counter = math.ceil(counter)
            end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
            counter -= 1

            data = {
                    "fontip": self.fon_type,
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

            info = self._do_post(data)
            info = info_schema.load(info)
            info = pd.DataFrame(info, columns=info_schema.fields.keys())
            info['fonunvantip'] = fonunvantip.replace("(","").replace(")","").replace(" ","")
            info = info[info['price'] != 0]
            if not info.empty :
                info_result = pd.concat([info_result, info])
                info_result = info_result.reset_index(drop=True)
                info = info.reset_index(drop=True)

            if counter > 0 :
                start_date = end_date + timedelta(days=1)
                end_date = end_date + timedelta(days=range_interval)
                if end_date > end_date_initial :
                    end_date = end_date_initial

        return info_result

    def fetch_info_serial(self, fonunvantips, start_date_initial, end_date_initial):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            if not info.empty :
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

        return merged

    def fetch_info_concurrently(self, fonunvantips, start_date_initial, end_date_initial):
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            self.concurrently = True
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                print(f"{fonunvantip} - {len(info)} records added total records: {len(merged)} " )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
        unvantip: bool = False,
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = [""]
        if unvantip :
            fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")

        if self.use_Proxy :
            proxy_address = self.get_free_proxy()
            self.proxies = {"http": proxy_address, "https": proxy_address}
            print(self.proxies)
        else :
            self.proxies = None

        if self.concurrently :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def get_free_proxy(self):
        proxy_address = FreeProxy(timeout=1, rand=True, https=True).get()
        return proxy_address

    def _do_post(self, data: Dict[str, str]) -> Dict[str, str]:
        timestamp = int(time.time() * 1000)  # Get current timestamp in milliseconds
        headers = {
         "Connection": "keep-alive",
         "Cache-Control": "no-cache",
         "Pragma": "no-cache",
         "X-Requested-With": "XMLHttpRequest",
         "Sec-Fetch-Mode": "cors",
         "Sec-Fetch-Site": "same-origin",
         "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
         "Accept": "application/json, text/javascript, */*; q=0.01",
         "Origin": "https://www.tefas.gov.tr",
         "Referer": f"https://www.tefas.gov.tr/TarihselVeriler.aspx?timestamp={timestamp}" ,
         }

        response = requests.post(
             url=f"{self.root_url}/{self.info_endpoint}",
             data=data,
             proxies=self.proxies,
             headers=headers,
         )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted



In [7]:
tefas = tefas_get()

time_delta = 90
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "market_cap", "number_of_shares", "number_of_investors" ], unvantip=False)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data.rename(columns={'price': 'close'}, inplace=True)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]

#fetched_data = fetched_data.drop(['fonunvantip','title'], axis=1)


 - 91214 records added total records: 91214 


,code,date,close,market_cap,number_of_shares,number_of_investors
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
91209,NaN,NaN,NaN,NaN,NaN,NaN
91210,NaN,NaN,NaN,NaN,NaN,NaN
91211,NaN,NaN,NaN,NaN,NaN,NaN
91212,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
fetched_data = fetched_data.sort_values(['code', 'date'])
fetched_data['open'] = fetched_data.groupby('code')['close'].shift(1)
fetched_data['high'] = fetched_data[['open', 'close']].max(axis=1)
fetched_data['low'] = fetched_data[['open', 'close']].min(axis=1)
fetched_data

,code,date,close,market_cap,number_of_shares,number_of_investors,open,high,low
89748,AAK,2024-05-21,21.713456,9.822622e+07,4523749.0,958.0,NaN,21.713456,21.713456
88282,AAK,2024-05-22,21.903943,1.002400e+08,4576344.0,972.0,21.713456,21.903943,21.713456
86814,AAK,2024-05-23,21.829808,1.007180e+08,4613781.0,980.0,21.903943,21.903943,21.829808
85345,AAK,2024-05-24,21.718660,1.013405e+08,4666055.0,989.0,21.829808,21.829808,21.718660
83876,AAK,2024-05-27,21.654609,1.015693e+08,4690425.0,995.0,21.718660,21.718660,21.654609
...,...,...,...,...,...,...,...,...,...
7612,ZZL,2024-08-13,46.507590,2.368720e+07,509319.0,15.0,46.522460,46.522460,46.507590
6084,ZZL,2024-08-14,46.679277,2.377464e+07,509319.0,15.0,46.507590,46.679277,46.507590
4557,ZZL,2024-08-15,46.681046,2.377554e+07,509319.0,15.0,46.679277,46.681046,46.679277
3030,ZZL,2024-08-16,46.776140,2.382398e+07,509319.0,15.0,46.681046,46.776140,46.681046


In [13]:
def calculate_rsi(group, window_length):
    rsi_values = ta.rsi(close=group['close'], length=window_length)
    group[f'rsi{window_length}'] = rsi_values  # Assign RSI values directly
    return group

window_length = 14
fetched_data = fetched_data.reset_index(drop=True)
fetched_data.sort_values(by=['code', 'date'], inplace=True)
fetched_data = fetched_data.groupby(['code']).apply(calculate_rsi, window_length=window_length)
fetched_data = fetched_data.fillna(0)
fetched_data[f'rsi{window_length}'].astype(float,False)
#fetched_data.to_excel("fetched_data.xlsx", index=False)
fetched_data

<ipython-input-13-5fe564097a4d>:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fetched_data = fetched_data.groupby(['code']).apply(calculate_rsi, window_length=window_length)


code        date      close    market_cap  number_of_shares  \
code                                                                     
AAK  0      AAK  2024-05-21  21.713456  9.822622e+07         4523749.0   
     1      AAK  2024-05-22  21.903943  1.002400e+08         4576344.0   
     2      AAK  2024-05-23  21.829808  1.007180e+08         4613781.0   
     3      AAK  2024-05-24  21.718660  1.013405e+08         4666055.0   
     4      AAK  2024-05-27  21.654609  1.015693e+08         4690425.0   
...         ...         ...        ...           ...               ...   
ZZL  91209  ZZL  2024-08-13  46.507590  2.368720e+07          509319.0   
     91210  ZZL  2024-08-14  46.679277  2.377464e+07          509319.0   
     91211  ZZL  2024-08-15  46.681046  2.377554e+07          509319.0   
     91212  ZZL  2024-08-16  46.776140  2.382398e+07          509319.0   
     91213  ZZL  2024-08-19  47.083847  2.398070e+07          509319.0   

            number_of_investors       open       high        low        rsi  \
code                                                                          
AAK  0                    958.0   0.000000  21.713456  21.713456   0.000000   
     1                    972.0  21.713456  21.903943  21.713456   0.000000   
     2                    980.0  21.903943  21.903943  21.829808   0.000000   
     3                    989.0  21.829808  21.829808  21.718660   0.000000   
     4                    995.0  21.718660  21.718660  21.654609   0.000000   
...                         ...        ...        ...        ...        ...   
ZZL  91209                 15.0  46.522460  46.522460  46.507590  75.841459   
     91210                 15.0  46.507590  46.679277  46.507590  78.100619   
     91211                 15.0  46.679277  46.681046  46.679277  78.123319   
     91212                 15.0  46.681046  46.776140  46.681046  79.361782   
     91213                 15.0  46.776140  47.083847  46.776140  82.762331   

                rsi14  
code                   
AAK  0       0.000000  
     1       0.000000  
     2       0.000000  
     3       0.000000  
     4       0.000000  
...               ...  
ZZL  91209  75.841459  
     91210  78.100619  
     91211  78.123319  
     91212  79.361782  
     91213  82.762331  

[91214 rows x 11 columns]

In [ ]:
fetched_data_agg = tefas.fetch(start=date_end, end=date_end, columns=["code", "date", "price", "fonunvantip", "title"], unvantip=True)
fon_table = fetched_data_agg.pivot(index=['code','title'], columns='fonunvantip', values='price').notnull()
del fetched_data_agg

In [ ]:
fon_table

,fonunvantip,AgresifDeğişken,Alternatif,Altın,BorçlanmaAraçları,DengeliDeğişken,Değişken,DiğerFonSepeti,Döviz,DövizCinsindenİhraçDolar,DövizCinsindenİhraçEuro,...,Sektör,Serbest,SürdürülebilirlikFonları,UzunVadeli,Yabancı,YabancıFonSepeti,ÇalışanlarınaYönelik,ÇokluVarlık,Özel,İştirak
code,title,,,,,,,,,,,,,,,,,,,,,
AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİSSE SENEDİ YOĞUN FON),False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABG,ATLAS PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZVB,ATLAS PORTFÖY SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
ZVO,ZİRAAT PORTFÖY ÜÇÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
ZYC,ZİRAAT PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [ ]:
fetched_data

code        date      close   market_cap  number_of_shares  \
code                                                                     
AAK  80882   AAK  2023-08-09  14.396935  90758436.12         6304011.0   
     79619   AAK  2023-08-10  14.577917  92342514.67         6334411.0   
     78356   AAK  2023-08-11  14.454878  91256854.61         6313222.0   
     77093   AAK  2023-08-14  14.759283  93083798.80         6306797.0   
     75829   AAK  2023-08-15  14.818671  93422741.08         6304394.0   
...          ...         ...        ...          ...               ...   
ZZL  348966  ZZL  2024-08-05  46.081096  23469977.87          509319.0   
     347448  ZZL  2024-08-06  46.547653  23707604.10          509319.0   
     345926  ZZL  2024-08-07  46.213071  23537195.22          509319.0   
     344401  ZZL  2024-08-08  46.352833  23608378.49          509319.0   
     342875  ZZL  2024-08-09  46.439744  23652643.81          509319.0   

             number_of_investors        rsi  
code                                         
AAK  80882                 625.0   0.000000  
     79619                 634.0   0.000000  
     78356                 637.0   0.000000  
     77093                 639.0   0.000000  
     75829                 645.0   0.000000  
...                          ...        ...  
ZZL  348966                 15.0  81.774054  
     347448                 15.0  86.310978  
     345926                 15.0  72.393688  
     344401                 15.0  74.260738  
     342875                 15.0  75.375999  

[348967 rows x 7 columns]

In [ ]:
fetched_data = fetched_data.reset_index(drop=True)
fon_table = fon_table.reset_index()
result = pd.merge(fetched_data, fon_table, on=['code'], how='left')

In [ ]:
result

,code,date,close,market_cap,number_of_shares,number_of_investors,rsi,title,AgresifDeğişken,Alternatif,...,Sektör,Serbest,SürdürülebilirlikFonları,UzunVadeli,Yabancı,YabancıFonSepeti,ÇalışanlarınaYönelik,ÇokluVarlık,Özel,İştirak
0,AAK,2023-08-09,14.396935,90758436.12,6304011.0,625.0,0.000000,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,...,False,False,False,False,False,False,False,True,False,False
1,AAK,2023-08-10,14.577917,92342514.67,6334411.0,634.0,0.000000,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,...,False,False,False,False,False,False,False,True,False,False
2,AAK,2023-08-11,14.454878,91256854.61,6313222.0,637.0,0.000000,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,...,False,False,False,False,False,False,False,True,False,False
3,AAK,2023-08-14,14.759283,93083798.80,6306797.0,639.0,0.000000,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,...,False,False,False,False,False,False,False,True,False,False
4,AAK,2023-08-15,14.818671,93422741.08,6304394.0,645.0,0.000000,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348962,ZZL,2024-08-05,46.081096,23469977.87,509319.0,15.0,81.774054,ZİRAAT PORTFÖY ONDÖRDÜNCÜ SERBEST FON,False,False,...,False,True,False,False,False,False,False,False,False,False
348963,ZZL,2024-08-06,46.547653,23707604.10,509319.0,15.0,86.310978,ZİRAAT PORTFÖY ONDÖRDÜNCÜ SERBEST FON,False,False,...,False,True,False,False,False,False,False,False,False,False
348964,ZZL,2024-08-07,46.213071,23537195.22,509319.0,15.0,72.393688,ZİRAAT PORTFÖY ONDÖRDÜNCÜ SERBEST FON,False,False,...,False,True,False,False,False,False,False,False,False,False
348965,ZZL,2024-08-08,46.352833,23608378.49,509319.0,15.0,74.260738,ZİRAAT PORTFÖY ONDÖRDÜNCÜ SERBEST FON,False,False,...,False,True,False,False,False,False,False,False,False,False


In [ ]:
!pip install openpyxl --quiet

In [ ]:
import openpyxl
result.to_excel("result.xlsx", index=False)